*Desenvolvimento*

In [2]:
import sys
import os
import mlflow
import mlflow.sklearn
from urllib.parse import urlparse

#Manipulação de dados
import pandas as pd

# Pré-Processamento
from sklearn.preprocessing import StandardScaler

# Criação do modelo
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

#Métricas
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

#Ignorar avisos de atualização, etc
import warnings
warnings.filterwarnings("ignore")

#Gráficos
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Código para sistema de log 

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [6]:
mlflow.set_tracking_uri('http://127.0.0.1:5000/')
mlflow.set_experiment(experiment_name='Wine_Classification')
tags = {
        "Projeto": "Tutorial MLflow",
        "team": "Data Science",
        "dataset": "Wine"
       }

In [31]:
def metricas(y_test, y_predict):
    acuracia = accuracy_score(y_test, y_predict)
    precision = precision_score(y_test, y_predict,average='weighted')
    recall = recall_score(y_test, y_predict,average='weighted')
    f1 = f1_score(y_test, y_predict,average='weighted')
    return acuracia, precision, recall, f1

In [32]:
def matriz_confusao(y_test, y_predict):
    matriz_conf = confusion_matrix(y_test.values.ravel(), y_predict)
    fig = plt.figure()
    ax = plt.subplot()
    sns.heatmap(matriz_conf, annot=True, cmap='Blues', ax=ax);

    ax.set_xlabel('Valor Predito');
    ax.set_ylabel('Valor Real'); 
    ax.set_title('Matriz de Confusão'); 
    ax.xaxis.set_ticklabels(['Classe 1', 'Classe 2', 'Classe 3']);
    ax.yaxis.set_ticklabels(['Classe 1', 'Classe 2', 'Classe 3']);
    plt.close()
    return fig

In [33]:
if __name__ == "__main__":
    warnings.filterwarnings("ignore")

    df = pd.read_csv('C:/Users/mymy/WORKSPACE/MLOps/wine-mlflow/data.csv', sep = ',',names=range(1, 15, 1))

    train, test = train_test_split(df, random_state=42)
    x_train = train.drop(columns=[1])
    x_test = test.drop(columns=[1])
    y_train = train[[1]]
    y_test = test[[1]]

In [39]:
import os
import mlflow
import mlflow.sklearn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

with mlflow.start_run(run_name='KNeighborsClassifier'):
    # Registro das tags
    mlflow.set_tags(tags)

    # Criação do modelo
    n_vizinhos = 5
    scaler = StandardScaler()
    knn = KNeighborsClassifier(n_neighbors=n_vizinhos)
    pipe = Pipeline([('scaler', StandardScaler()), ('knn', knn)])
    pipe.fit(x_train, y_train)

    # Predição dos valores de teste
    y_pred_knn = pipe.predict(x_test)

    # Métricas
    acuracia, precision, recall, f1 = metricas(y_test, y_pred_knn)
    print("Número de Vizinhos: {}\nAcurácia: {}\nPrecision: {}\nRecall: {}\nF1-Score: {}".format(
        n_vizinhos, acuracia, precision, recall, f1))

    # Matriz de confusão
    matriz_conf = matriz_confusao(y_test, y_pred_knn)
    temp_name = "confusion-matrix.png"
    matriz_conf.savefig(temp_name)  # Salva a matriz de confusão como imagem
    mlflow.log_artifact(temp_name, "confusion-matrix-plots")  # Log da imagem no MLflow
    try:
        os.remove(temp_name)
    except FileNotFoundError as e:
        print(f"{temp_name} file is not found")

    # Registro dos parâmetros e métricas
    mlflow.log_param("n_neighbors", n_vizinhos)
    mlflow.log_metric("Acuracia", acuracia)
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("Recall", recall)
    mlflow.log_metric("F1-Score", f1)

    # Registro do modelo
    mlflow.sklearn.log_model(pipe, "modelo_wine_classificador")


Número de Vizinhos: 5
Acurácia: 0.9555555555555556
Precision: 0.9586538461538462
Recall: 0.9555555555555556
F1-Score: 0.9550512333965844


2025/06/06 19:31:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run KNeighborsClassifier at: http://127.0.0.1:5000/#/experiments/574827219160013650/runs/6e1a13593fc848af9c7aeda19fbbbc8f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/574827219160013650


In [ ]:
import mlflow
import pandas as pd

mlflow.set_tracking_uri('http://127.0.0.1:5000/')

#Carregando o modelo que está em produção
logged_model = 'models:/tutorial_mlflow/Production'
loaded_model = mlflow.pyfunc.load_model(logged_model)